In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import seaborn as sns
sns.set()
from sklearn.cluster import KMeans


C:\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
import cx_Oracle
print(cx_Oracle.version)
#check the Oracle Version using: select * from v$version
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)
#follow this link to solove issue: cx_Oracle error. DPI-1047: Cannot locate a 64-bit Oracle Client library
# https://stackoverflow.com/questions/56119490/cx-oracle-error-dpi-1047-cannot-locate-a-64-bit-oracle-client-library?answertab=trending#tab-top
import os
import platform
# This is the path to the ORACLE client files
lib_dir = r"C:\Users\adam.blaine\Documents\instantclient-basic-windows.x64-21.6.0.0.0dbru\instantclient_21_6"

# Diagnostic output to verify 64 bit arch and list files
print("ARCH:", platform.architecture())
print("FILES AT lib_dir:")
name_string = ""
for name in os.listdir(lib_dir):
    name_string = name_string + name + " "
print(name_string)

#you just need to run the following snipet for once, thats why I comment them out since I have ran this    
try:
    cx_Oracle.init_oracle_client(lib_dir=lib_dir)
except Exception as err:
    print("Error connecting: cx_Oracle.init_oracle_client()")
    print(err);
    sys.exit(1);

cx_Oracle.clientversion()

8.3.0
C:\Anaconda3\python.exe
3.6.1 |Anaconda 4.4.0 (64-bit)| (default, May 11 2017, 13:25:24) [MSC v.1900 64 bit (AMD64)]
sys.version_info(major=3, minor=6, micro=1, releaselevel='final', serial=0)
ARCH: ('64bit', 'WindowsPE')
FILES AT lib_dir:
adrci.exe adrci.sym BASIC_LICENSE BASIC_README genezi.exe genezi.sym network oci.dll oci.sym ocijdbc21.dll ocijdbc21.sym ociw32.dll ociw32.sym ojdbc8.jar oramysql.dll oramysql.sym orannzsbb.dll orannzsbb.sym oraocci21.dll oraocci21.sym oraocci21d.dll oraocci21d.sym oraociei.dll oraociei.sym orasql.dll orasql.sym ucp.jar uidrvci.exe uidrvci.sym xstreams.jar 


(21, 6, 0, 0, 0)

In [3]:
from sqlalchemy.engine import create_engine
import numpy as np
DIALECT = 'oracle'
SQL_DRIVER = 'cx_oracle'
USERNAME = "ablaine" #enter your username
PASSWORD = "fUtk!n8MgYGSKUnJ" #enter your password
HOST = "p9cpwpjdadb01" #enter the oracle db host url
PORT = 25959 # enter the oracle port number
SERVICE = "FR01PR" # enter the oracle db service name
ENGINE_PATH_WIN_AUTH = DIALECT + '+' + SQL_DRIVER + '://' + USERNAME + ':' + PASSWORD +'@' + HOST + ':' + str(PORT) + '/?service_name=' + SERVICE

engine = create_engine(ENGINE_PATH_WIN_AUTH)

#test query

db = pd.read_sql_query('''
Select i.U_CATEGORY,i.U_COMMODITY, i.u_style, i.u_choice, i.U_SPECIFICCOLORNAME, i.U_SIZE_ONE, R.ITEM, r.source, r.dest, r.QTY, R.ORDERPLACEDATE,r.SCHEDARRIVDATE, s.OH ,dmd.u_permretailprice retail
, (SELECT SUM(nvl(f.qty,0)) FROM scpomgr.cds_fcstview f where f.dmdunit = s.item and f.loc = s.loc and f.startdate between sysdate - 7 and sysdate +42) "total fcst"
from SCPOMGR.recship r
inner join SCPOMGR.item i on i.item = r.item
INNER JOIN SCPOMGR.SKU S ON S.ITEM = r.ITEM AND S.LOC = r.dest
inner join SCPOMGR.dmdunit dmd on i.item = dmd.dmdunit
where
i.u_style in ('S_1200BZ',
'S_28116A',
'S_28117A',
'S_28118A',
'S_28154A',
'S_28155A',
'S_67181',
'S_67182',
'S_67479-HH',
'S_67482',
'S_6ARCDHASCBOGOF1',
'S_6ARCDHASCBOGOF9',
'S_6ARDHHAS1064053',
'S_6AREDHASHBSPTS2',
'S_75729',
'S_79645',
'S_79646'
)
and R.source IN ('000155','000350')--'V100033'
--and r.dest in ('000357','000534','000536','000540') 
and R.ORDERPLACEDATE between TO_DATE('2022-07-03','YYYY-MM-DD') and TO_DATE('2022-08-03','YYYY-MM-DD') 
''', engine)

#dont forget to close the engine to release the resource you have used on the server.
engine.dispose()

In [4]:
db.columns = db.columns.str.upper()

In [5]:
db.sort_values(by=['DEST',  'ITEM','ORDERPLACEDATE'], ascending = [True, True, True], inplace=True )
db = db.reset_index(drop=True)

In [6]:
db['GAP'] = db.groupby(['DEST', 'ITEM'])['ORDERPLACEDATE'].diff()


In [7]:
db['QTY'] = db['QTY'].astype('int64')
db['OH'] = db['OH'].astype('int64')
db['total fcst'] = db['total fcst'].astype('int64')


In [8]:
db['GAP'] =db['GAP'].fillna(0)
# db.fillna(pd.Timedelta('0 days')) #For newer versions of pandas

In [9]:
db.GAP = (db.GAP / np.timedelta64(1,'D')).astype(int)


In [10]:
db['SUPPLY'] =  round(db['OH'] / ((db['total fcst']+0.0001) / 7) , 2)


In [11]:
db['STATUS'] = 'Original'


In [12]:

db['2W Prior'] = np.where((db.loc[:,'GAP']< 14) & (db.loc[:,'GAP'] > 0), True, False)
db['1W Prior'] = np.where((db['GAP']< 8) & (db.loc[:,'GAP'] > 0), True, False)
db['2W lowQ'] = np.where((db['GAP']< 14) & (db['QTY'] < 4) & (db.loc[:,'GAP'] > 0), True, False)
db['1W lowQ'] = np.where((db['GAP']< 8) & (db['QTY'] < 4) & (db.loc[:,'GAP'] > 0), True, False)


In [13]:

db1 = db.copy()
for i in range (1, len(db1)):
    if (db1.loc[i, '1W lowQ'] == True) & (db1.loc[i - 1, '1W lowQ'] == True):
        db1.loc[i, '1W lowQ'] = False 


In [14]:

for i in range (1, len(db1)):
    if (db1.loc[i, '1W lowQ'] == True) :
        if db1.loc[i, 'SUPPLY'] < 10 :
            db1.loc[i-1 , 'QTY'] = db1.loc[i-1 , 'QTY'] + db1.loc[i, 'QTY']
            db1.loc[i , 'QTY'] = 0
            db1.loc[i-1, 'STATUS'] = 'Revised'
            db1.loc[i, 'STATUS'] = 'Removed'
            
        else:
            db1.loc[i, 'QTY'] = db1.loc[i-1 , 'QTY'] + db1.loc[i, 'QTY']
            db1.loc[i-1 , 'QTY'] = 0
            db1.loc[i, 'STATUS'] = 'Revised'
            db1.loc[i-1, 'STATUS'] = 'Removed'


In [15]:

for i in range (1, len(db1)):
    if (db1.loc[i, '2W Prior'] == True) & (db1.loc[i, 'SUPPLY'] < 3) & (db1.loc[i, 'STATUS'] == 'Original') :
        db1.loc[i - 1, 'QTY'] = db1.loc[i-1 , 'QTY'] = db1.loc[i-1 , 'QTY'] + db1.loc[i, 'QTY']
        db1.loc[i , 'QTY'] = 0
        db1.loc[i-1, 'STATUS'] = 'Revised'
        db1.loc[i, 'STATUS'] = 'Removed'


In [16]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('Revised_RecShip.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
db1.to_excel(writer, sheet_name='revised')
db.to_excel(writer, sheet_name='orig')

# Close the Pandas Excel writer and output the Excel file.
writer.save()